# Bangumi Takeout More

> 与主笔记本的区别：本笔记本旨在备份小组讨论、日志与目录，并主要以网页而非 API 形式与 Bangumi 交互，需要 Cookies 而非 API Token。

项目主页：[jerrylususu/bangmui-takeout-py](https://github.com/jerrylususu/bangumi-takeout-py)


# <font color='red'>重要安全警告</font>

<font color='red'>⚠️ 如果在 2023/3/7 前使用过本工具，请务必阅读！</font> 

如果你在 2023/3/7 之前使用过本工具，你可能已经安装了 Get Cookies.txt 插件。这一插件最近被发现为恶意插件。请尽快遵循以下指引以最小化损失：
如果你在 2023/3/7 之前使用过 Bangumi Takeout More，你可能已经安装了 Get Cookies.txt 插件。这一插件最近被发现为恶意插件。请尽快遵循以下指引以最小化损失：
1. 首先，卸载扩展；
2. 有些网站会显示已登录设备，并提供注销的选择，你可以在那个页面注销已登录设备；
    - Google: 管理您的 Google 账号 - 安全性 - 管理所有设备
    - Github: Settings - Assess - Sessions
    - Microsoft: 我的 Microsoft 账户 - 安全 - 登录活动
3. 最有效的防御手段是：浏览你的 cookie 列表，依次打开网站，主动点击注销按钮，网站会收到你的注销请求然后吊销该 cookie，保证开发者拥有无效数据；
4. 最后的最后，如果还不放心，请修改密码，部分网站在修改密码后会吊销所有 cookie。

[Bangumi 上的相关讨论](https://bgm.tv/group/topic/378915)

直到找到更安全的获取 Cookies 方式前，请不要使用本项目进行导出！

## 安全声明
本笔记本使用 Cookies 与 Bangumi 交互。所有请求都为读请求，没有任何写操作。为保护数据安全，推荐在运行完本脚本后手动在 Bangumi 主站登出再登入以无效化提交至脚本的 Cookie。

## 运行步骤

1. 请点击菜单栏「代码执行程序 - 全部运行」（繁體中文「執行階段－全部執行」），随后卷动到页面底部，开始填写执行本脚本所需的信息。
2. `User-Agent` 字符串：点击提供的链接，将页面上半部分文本框内的内容复制并填入。（通常以 `Mozilla/5.0` 开始）
3. `User ID`：打开 Bangumi 首页，点击右上角个人头像进入个人主页，个人主页的网址的最后一部分即为 User ID。若未自定义通常为数字。
4. `Cookie`：打开 Bangumi 首页，按下 F12 打开「开发者工具」，在「控制台」中输入以下指令：`console.log(document.cookie)`，复制输出的内容即可。通常为较长的，包含 `chii` 字样的字符串。
5. 选择导出内容，以及是否要深度导出。不勾选深度导出时，导出内容仅为讨论/日志/目录的列表，包含如标题、ID、创建时间、URL 等元数据，但不含具体的数据。勾选深度导出则会同时导出元数据和具体数据（讨论和日志以 HTML 网页形式，目录同时以 HTML 网页和 JSON 形式）。
6. 最后点击「运行」以启动导出过程。执行完成会自动触发 `dump.zip` 的下载。



## 常见问题

表单可能需要稍等一会才会显示（需要等待运行时分配，进度会在右上角显示），请耐心等待，最长应该不会超过一分钟。

为了避免对 Bangumi 服务器造成过大压力，每个网络请求之间会间隔一秒，还请耐心等待。

理论上可以在 `User ID` 中填入他人的 User ID，可以备份他人公开的项目。但此种用法未经测试，故不推荐。

In [ ]:
from ipywidgets import interact, widgets
from IPython.display import display
from google.colab import files

display(widgets.HTML(value='<h3>运行参数</h3>'))


ua_html = widgets.HTML(value='获取 User-Agent：<a href="https://www.whatsmyua.info/" target="_blank">点击此链接，并复制文本框中的 User-Agent 字符串</a>')
display(ua_html)

ua = widgets.Text(value='',placeholder='将 User-Agent 字符串粘贴到此处!',description='User-Agent:',disabled=False)
display(ua)

display(widgets.HTML(value='获取 User ID：打开 <a href="https://www.bgm.tv/" target="_blank">Bangumi 首页</a>，点击右上角头像进入个人主页，网址最后的部分即为 User ID （https://bgm.tv/user/{user_id}）<br> 通常为 6 位数字。注意不是用户昵称！'))
user_id = widgets.Text(value='',placeholder='通常为数字，不是昵称，不包含@',description='User ID:',disabled=False)
display(user_id)

display( widgets.HTML(value='获取 Cookie：打开 <a href="https://www.bgm.tv/" target="_blank">Bangumi 首页</a>，按下 F12 打开「开发者工具」，在「控制台」中输入以下指令后按下回车：<span style="font-family: monospace">  console.log(document.cookie)  </span>，复制输出的内容即可。'))
cookie_str = widgets.Text(value='',placeholder='很长的字符串，包含 chii_',description='Cookie:',disabled=False)
display(cookie_str)

display(widgets.HTML(value='<hr> <h3>备份内容</h3> 作者个人建议：先不勾选「深度备份」，备份一次基础数据；然后再对每个项目单独勾选深度备份。'))

topic = widgets.Checkbox(value=True, description='我发表的讨论', indent=True)
display(topic)
reply_topic = widgets.Checkbox(value=True, description='我回复的讨论', indent=True)
display(reply_topic)
blog = widgets.Checkbox(value=True, description='日志', indent=True)
display(blog)
created_index = widgets.Checkbox(value=True, description='创建的目录', indent=True)
display(created_index)
collected_index = widgets.Checkbox(value=True, description='收藏的目录', indent=True)
display(collected_index)
timeline = widgets.Checkbox(value=True, description='时间胶囊（备份过程比较慢）', indent=True)
display(timeline)
person = widgets.Checkbox(value=True, description='收藏的人物（虚拟&现实）', indent=True)
display(person)
friend = widgets.Checkbox(value=True, description='好友', indent=True)
display(friend)


display(widgets.HTML(value='深度备份（不仅备份讨论/日志/收藏列表，也备份对应的网页）'))

deep = widgets.Checkbox(value=False, description='深度备份', indent=True)
display(deep)



button = widgets.Button(tooltip='运行',description="运行")
display(button)

def callback(wdgt):

    !git clone --depth=1 https://github.com/jerrylususu/bangumi-takeout-py 
    !cd bangumi-takeout-py && git log -n1 --format="commit: %h"
    !mv bangumi-takeout-py/* .
    !rm -rf bangumi-takeout-py
    !pip install -r requirements.txt
    
    import dump_personal

    dump_personal.main(user_id=user_id.value, 
                       user_agent=ua.value,
                       cookie_str=cookie_str.value, 
                       topic=topic.value, 
                       reply_topic=reply_topic.value,
                       blog=blog.value, 
                       created_index=created_index.value, 
                       collected_index=collected_index.value,
                       timeline=timeline.value,
                       person=person.value,
                       friend=friend.value, 
                       deep=deep.value)
  
    files.download("dump.zip")

button.on_click(callback)